In [2]:
from openai import OpenAI
import numpy as np

import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()
EMBED_MODEL = "text-embedding-3-small"

#FAQ 모음
FAQS = [
    {"id": 1, "title": "결제 수단 변경", "content": "결제 수단은 계정 설정의 결제 정보에서 변경할 수 있습니다."},
    {"id": 2, "title": "환불 규정", "content": "결제 후 7일 이내에는 전액 환불이 가능합니다
]

def embed(text: str):
    return client.embeddings.create(
        model=EMBED_MODEL,
        input=text
    ).data[0].embedding

for faq in FAQS:
    faq["embedding"] = np.array(embed(faq["content"]), dtype="float32")

print(FAQS)



[{'id': 1, 'title': '결제 수단 변경', 'content': '결제 수단은 계정 설정의 결제 정보에서 변경할 수 있습니다.', 'embedding': array([ 0.01855729, -0.00994543, -0.0191959 , ..., -0.02261435,
       -0.00141809, -0.00354758], dtype=float32)}, {'id': 2, 'title': '환불 규정', 'content': '결제 후 7일 이내에는 전액 환불이 가능합니다.', 'embedding': array([-0.04589606,  0.04492525, -0.05446601, ..., -0.01074591,
       -0.0084737 ,  0.00603411], dtype=float32)}]


In [4]:
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def retrieve(q, k=1):
  q_emb = np.array(embed(q), dtype="float32")
  scored = []
  for faq in FAQS:
    score = cosine_sim(q_emb[0], faq["embedding"][0])
    scored.append({"id": faq["id"], "score": score, "content": faq["content"]})

  scored = sorted(scored, key=lambda x: x["score"], reverse=True)
  return scored[:k]

# print(retrieve("결제 후 환불은 언제까지 가능해?"))
print(retrieve("카드 말고 페이머니로 결제하려면 어떻게 해야 할까?"))

[{'id': 1, 'score': np.float32(1.0), 'content': '결제 수단은 계정 설정의 결제 정보에서 변경할 수 있습니다.'}]


In [7]:
def answer_with_rag(question: str, k=3):
    top_faqs = retrieve(question, k=k)
    context = "\n\n".join(
        [f"[DOC {faq['id']}] {faq['content']}" for faq in top_faqs]
    )

    print("context = ", context)

    instructions = """
    너는 고객센터 상담사야.
    아래 제공된 문서 조각(context)을 우선적으로 참고해서 사용자의 질문에 답해라.
    context에 없으면 모른다고 답해라.
    답변 마지막에 참고한 문서 ID를 대괄호로 표시해라. 예: [DOC 1, DOC 3]
    """

    resp = client.responses.create(
        model="gpt-4o-mini",
        instructions=instructions,
        input=f"사용자 질문: {question}\n\n[context]\n{context}",
        max_output_tokens=256,
        temperature=0.2,
    )
    print(resp.output_text)
    return resp

resp = answer_with_rag("현금으로 결제하는 것도 가능해?", 1)

context =  [DOC 1] 결제 수단은 계정 설정의 결제 정보에서 변경할 수 있습니다.
죄송하지만, 제공된 문서에는 현금 결제 가능 여부에 대한 정보가 없습니다. 추가적인 정보는 확인할 수 없습니다. [DOC 1]


In [12]:
from openai import OpenAI


client = OpenAI()

response = client.responses.create(
  model="gpt-4o-mini",
  instructions ="너는 고객센터 상담사야. 아래 질문에 답변해 줘.",
  input="결제 후 환불은 언제까지 가능해? 전액이야, 반액이야?.",
  top_p=0.2,

)

print(response.output_text)


결제 후 환불 가능 기간은 보통 구매한 상품이나 서비스의 종류에 따라 다릅니다. 일반적으로는 결제일로부터 7일 이내에 요청하시면 전액 환불이 가능하며, 이후에는 반액 환불이 적용될 수 있습니다. 하지만 특정 상품이나 서비스에 따라 다를 수 있으니, 구체적인 환불 정책은 구매하신 곳의 약관을 확인하시거나 고객센터에 문의해 주시면 정확한 정보를 안내해 드리겠습니다.
